In [1]:
import pandas as pd
import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt

import folium
from folium.plugins import FastMarkerCluster

## Geocode the addresses from Real Estate transactions database
Data taken from NYdatabase.com

In [57]:
locator = Nominatim(user_agent="my_Geocoder")
location = locator.geocode("911 East Shore Drive, Tompkins County, NY, USA")

In [3]:
print(location.address)
print("Latitude ={}, Longitude ={}".format(location.latitude, location.longitude))    # Test location Lat-Long checks out

911, East Shore Drive, Renwick, Ithaca Town, Tompkins County, New York, 14850, United States of America
Latitude =42.46469782009001, Longitude =-76.49928169371793


In [15]:
df = pd.read_csv("real_est.csv")

In [18]:
del df['index']

In [19]:
df

,City,Address,Zip,Price,Date,Seller,Buyer,Transaction Type
0,Ithaca,"911 East Shore Drive(Ithaca, 14850)",14850,"$128,000",July-09-2020,"Wirtz, Trustee, Mary H.","Card, William T.",Standard (arm's length)
1,Ithaca,"114 Kelvin Place(Ithaca, 14850)",14850,"$610,000",July-09-2020,"Harrell, James S.","Verma, Sushil K.",Standard (arm's length)
2,Ithaca,"326 Coddington Road(Ithaca, 14850)",14850,"$157,000",July-09-2020,"Matthew Moghaddam, Estate","Pieper, Alphonse",Standard (arm's length)
3,Ithaca,"730 Cliff Street(Ithaca, 14850)",14850,"$180,000",July-06-2020,"Erickson, Brian G.","Cuevas Gongora, Joaquin Armando",Standard (arm's length)
4,Ithaca,"Third Street(Ithaca, 14850)",14850,$1,July-06-2020,Cornell University,"The People of the State, of New York","Not a sale deed, Governement sale"
...,...,...,...,...,...,...,...,...
495,Ithaca,"317 Elm St(Ithaca, 14850)",14850,"$269,000",June-26-2019,"Rhodes, David D","Cederstrom, Arthur",Standard (arm's length)
496,Ithaca,"212 Fall Creek Dr(Ithaca, 14850)",14850,"$500,000",June-26-2019,"Fraser, Hilary T","Kysel, Ian M",Interest conveyed
497,Ithaca,"1089 & 0 Taughannock Blvd(Ithaca, 14850)",14850,$1,June-26-2019,"Burns, Joseph A, Exec, Estate of Judith Burns","Burns, Joseph A, TTEE, Judith K Burns Disclaimer","Buyer is seller, Not a sale deed, Sale between..."
498,Ithaca,"209 Eldridge Cir(Ithaca, 14850)",14850,"$525,000",June-26-2019,"Myers, Sharon A","Levy, Stephen L",Standard (arm's length)


### Drop govt sales, btw relatives or nominal sum
Drop govt related sales, transfers between relatives (or sale to self), or sale price of $1

In [21]:
df['Transaction Type'].unique()

array(["Standard (arm's length)", 'Not a sale deed, Governement sale',
       'Not a sale deed, Sale between relatives', 'Not a sale deed', nan,
       'Buyer is seller, Not a sale deed', 'Buyer is seller',
       'Related company sale', 'Sale between relatives',
       'Buyer is seller, Not a sale deed, Sale between relatives',
       'Buyer is seller, Not a sale deed, Multiple SWIS, Sale between relatives',
       'Buyer is seller, Sale between relatives', 'Governement sale',
       'Buyer is seller, Multiple SWIS',
       'Related company sale, Not a sale deed', 'Interest conveyed',
       'Buyer is seller, Related company sale, Not a sale deed',
       'Not a sale deed, Interest conveyed',
       'Buyer is seller, Related company sale',
       "Standard (arm's length), Multiple SWIS",
       'Multiple SWIS, Sale between relatives'], dtype=object)

In [26]:
df = df[df['Transaction Type']!= 'Not a sale deed, Governement sale']

In [29]:
df = df[~df['Transaction Type'].isna()]

In [ ]:
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
df = df[df['Price']!= 1.0]

In [41]:
df['Price'].describe().astype(int)

count        344
mean      360694
std       594522
min         1000
25%       196725
50%       280844
75%       375000
max      9529700
Name: Price, dtype: int64

In [44]:
df = df[~df['Transaction Type'].str.contains('Sale between relatives')]

In [47]:
df.head(5)

,City,Address,Zip,Price,Date,Seller,Buyer,Transaction Type
0,Ithaca,"911 East Shore Drive(Ithaca, 14850)",14850,128000.0,July-09-2020,"Wirtz, Trustee, Mary H.","Card, William T.",Standard (arm's length)
1,Ithaca,"114 Kelvin Place(Ithaca, 14850)",14850,610000.0,July-09-2020,"Harrell, James S.","Verma, Sushil K.",Standard (arm's length)
2,Ithaca,"326 Coddington Road(Ithaca, 14850)",14850,157000.0,July-09-2020,"Matthew Moghaddam, Estate","Pieper, Alphonse",Standard (arm's length)
3,Ithaca,"730 Cliff Street(Ithaca, 14850)",14850,180000.0,July-06-2020,"Erickson, Brian G.","Cuevas Gongora, Joaquin Armando",Standard (arm's length)
5,Ithaca,"311 E. Tompkins Street(Ithaca, 14850)",14850,179000.0,July-06-2020,"Lapp, Julia L.","Moore, William L.",Standard (arm's length)


### Clean Address Column to show just street name

In [48]:
df['Address'] = df['Address'].str.split('(').str[0] 

In [53]:
df['ADDRESS'] = df['Address'] + ', Tompkins County, NY, USA'

In [55]:
df.head(5)

,City,Address,Zip,Price,Date,Seller,Buyer,Transaction Type,ADDRESS
0,Ithaca,911 East Shore Drive,14850,128000.0,July-09-2020,"Wirtz, Trustee, Mary H.","Card, William T.",Standard (arm's length),"911 East Shore Drive, Tompkins County, NY, USA"
1,Ithaca,114 Kelvin Place,14850,610000.0,July-09-2020,"Harrell, James S.","Verma, Sushil K.",Standard (arm's length),"114 Kelvin Place, Tompkins County, NY, USA"
2,Ithaca,326 Coddington Road,14850,157000.0,July-09-2020,"Matthew Moghaddam, Estate","Pieper, Alphonse",Standard (arm's length),"326 Coddington Road, Tompkins County, NY, USA"
3,Ithaca,730 Cliff Street,14850,180000.0,July-06-2020,"Erickson, Brian G.","Cuevas Gongora, Joaquin Armando",Standard (arm's length),"730 Cliff Street, Tompkins County, NY, USA"
5,Ithaca,311 E. Tompkins Street,14850,179000.0,July-06-2020,"Lapp, Julia L.","Moore, William L.",Standard (arm's length),"311 E. Tompkins Street, Tompkins County, NY, USA"


### Apply geocode using address

In [58]:
# Delay between geocoding calls (1sec), convenient when geocoding large no. of addresses, as geocoding svc provider can deny access to service

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)    

# create location column & apply geocoding    

df['location'] = df['ADDRESS'].apply(geocode)   

# create, Lat, Long, Altitude as a single tuple column

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# split point colmn to individual columns

df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [61]:
df = df.drop(['City', 'Zip', 'altitude'], axis=1)

In [71]:
df = df[~df['Price'].isnull()]    # Drop null Price values

In [75]:
df[df['latitude'].isnull()].shape    # Problem with 48 addresses unable to geocode

(48, 11)

In [83]:
df_temp = df[~df['latitude'].isnull()]

### Map out the points

In [97]:
map2 = folium.Map(location = [42.4440, -76.5019],
                 tiles= 'cartodbpositron',
                 zoom_start=12)

In [98]:
df_temp.apply(lambda row: folium.Circle(location=[row['latitude'], row['longitude']], radius=2, color='crimson').add_to(map2), axis=1)

0      <folium.vector_layers.Circle object at 0x13a54...
1      <folium.vector_layers.Circle object at 0x13a54...
2      <folium.vector_layers.Circle object at 0x13a54...
3      <folium.vector_layers.Circle object at 0x13a54...
5      <folium.vector_layers.Circle object at 0x13a54...
                             ...                        
494    <folium.vector_layers.Circle object at 0x13a58...
495    <folium.vector_layers.Circle object at 0x13a58...
496    <folium.vector_layers.Circle object at 0x13a58...
498    <folium.vector_layers.Circle object at 0x13a58...
499    <folium.vector_layers.Circle object at 0x13a58...
Length: 287, dtype: object

In [99]:
map2

In [101]:
map2.save('real_est_ithaca.html')